<a href="https://colab.research.google.com/github/joeban0608/python_liontravel_project/blob/main/liontravel_download_csv_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# get_godatastart & godataend dates 
# 想要出發的行程區間,預設[當日]至[當日再往後加30日]
# 參考來源:https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/369869/
import datetime
import time
from datetime import timedelta, date


# 取[今日的str日期]方式
def get_liontravel_datas():
    def strTodatetime(datestr, format):
        return datetime.datetime.strptime(datestr, format)
    data_godatestart = time.strftime("%Y-%m-%d", time.localtime())

    # 取[未來30天的日期]方式
    def get_day_of_day(n=0):
        if(n<0):
            n = abs(n)
            return date.today()-timedelta(days=n)
        else:
            return date.today()+timedelta(days=n)
    data_godateend = str(get_day_of_day(30))
    return data_godatestart , data_godateend

In [ ]:
# get_fulltext
import requests
from bs4 import BeautifulSoup
import pandas as pd

data_godatestart= get_liontravel_datas()[0]
data_godateend = get_liontravel_datas()[1]

def Download_liontravel(input_SortType=1, input_Keywords=0,data_godatestart=data_godatestart,data_godateend=data_godateend):

# data_godatestart = "2021-02-26"
# data_godateend = "2021-03-28"

# input_SortType=1 依售價低到高排序 預設
# input_SortType=2 依售價高到低排序　 
# input_SortType=3 依出發日期排序　 
# input_SortType=4 依行程天數低到高排序　 
# input_SortType=5 依行程天數高到低排序　

# input_Keywords=0 關鍵字條件:一日遊  預設
# input_Keywords=0 關鍵字條件:台東
# input_Keywords=0 關鍵字條件:宜蘭
# input_Keywords=0 關鍵字條件:基隆

    table = {
        '編號':[],
        '行程名稱':[],
        '旅程天數':[],
        '價格':[],
        '訂購連結':[]
        }

    list_KEYWORDS = ["一日遊","台東","宜蘭","基隆"]
    PAGE = 1
    i = 0
    while True:
        url = "https://travel.liontravel.com/search/grouplist"
        GODATESTART = data_godatestart
        GODATEEND = data_godateend
        SORTYPE = str(input_SortType)

        KEYWORDS = list_KEYWORDS[input_Keywords]
        datas = {
                "ArriveID":",",
                "GoDateStart": GODATESTART,
                "GoDateEnd": GODATEEND,
                "Keywords": KEYWORDS,
                "IsEnsureGroup": "false",
                "IsSold": "true",
                "GoStartDays": "0",
                "TravelType": "1",
                "TourArea": "0",
                "Page": str(PAGE + i),
                "SortType": SORTYPE
                }
        if not datas["Page"] == "1":
           url = url + "item" 
        response = requests.post(url, datas)
        html = BeautifulSoup(response.text)
        travels = html.find_all("div",class_="rli_tlin")
        if not travels:
            break;
        for travel in travels:
            title = travel.find("div",class_="rli_body").find("div",class_="rli_content").find("a")

            price = travel.find("div",class_="rli_footer").find("div",class_="rli_price").find("a").find("div",class_="rli_pr_info").find("div",class_="price")

            link = "https://travel.liontravel.com" + title["href"]

            num = travel.find("div",class_="rli_body").find("div",class_="rli_view").find("div",class_="rli_tags").find("span",class_="circle_tag")

            day = travel.find("div",class_="rli_footer").find("div",class_="rli_day").find("span",class_="rli_font")
            day = day.text + "天"
            print("編號:" + num.text)
            print(title.text)
            print("旅程天數：",day)
            print(price.text)
            print("https://travel.liontravel.com/" + link)
            print("-" * 30)

            table['編號'].append(num.text)
            table['行程名稱'].append(title.text)
            table['旅程天數'].append(day)
            table['價格'].append(price.text)
            table['訂購連結'].append(link)
        i += 1

    df = pd.DataFrame(table)
    df.to_csv("liontravel_Search_"+ list_KEYWORDS[input_Keywords] +".csv", encoding="utf-8",  index=False)


In [ ]:
# 將上述兩個function 結合之後,只要變動downlaod_liontravel 的函數即可
Download_liontravel(input_SortType=1, input_Keywords=0,data_godatestart=data_godatestart,data_godateend=data_godateend)

# data_godatestart = 輸入行程出發時間  [預設今日日期]
# data_godateend  = 輸入行程結束時間  [預設今日日期+30日]

# input_SortType=1 依售價低到高排序 [預設]
# input_SortType=2 依售價高到低排序　 
# input_SortType=3 依出發日期排序　 
# input_SortType=4 依行程天數低到高排序　 
# input_SortType=5 依行程天數高到低排序　

# input_Keywords=0 關鍵字條件:一日遊 [預設]
# input_Keywords=1 關鍵字條件:台東
# input_Keywords=2 關鍵字條件:宜蘭
# input_Keywords=3 關鍵字條件:基隆
